In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
import yfinance as yf
import keras 
import os
import datetime 
from sklearn.preprocessing import MinMaxScaler

In [20]:
def TickerIntoTensor(ticker='aapl',daysago=100,daysforward=1,split_percentage=0.9):
    df = yf.Ticker(ticker.upper())
    df = df.history(period='max')
    if df.shape[1] == 7:
        df = df.drop(columns=['Dividends','Stock Splits'])
    df = df.drop(columns=['Open','High','Low','Volume'])
    for i in range(daysago,-daysforward-1,-1):
        df['{}daysago'.format(i)] = df['Close'].shift(i)
        df = df.copy()
    df = df.drop(columns=['Close'])
    df = df.drop(index=df.tail(daysforward).index)
    df = df.drop(index=df.head(daysago).index)
    dfstddev = df.iloc[:,:daysago+1].std(axis=1)
    sc = MinMaxScaler(feature_range=(0,1))
    stddev_scaled = sc.fit_transform(dfstddev.values.reshape(-1,1))
    print(stddev_scaled)
    df_scaled = sc.fit_transform(df)
    X,Y = df_scaled[:,:daysago+1],df_scaled[:,-daysforward:]
    if split_percentage <= 1 and split_percentage >= 0:
        split = int(len(X)*split_percentage)
    else:
        split = split_percentage
    X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]
    X_train, X_test = np.append(X_train,stddev_scaled[:split],axis=1), np.append(X_test,stddev_scaled[split:],axis=1)
    X_train, X_test, Y_train, Y_test = X_train.reshape((-1,daysago+1+1,1)), X_test.reshape((-1,daysago+1+1,1)), Y_train.reshape((-1,daysforward)), Y_test.reshape((-1,daysforward))

    return X_train, X_test, Y_train, Y_test
TickerIntoTensor('aapl',100,1,-1)

[[4.08300297e-04]
 [4.08082143e-04]
 [4.08033925e-04]
 ...
 [8.08066238e-01]
 [8.30667302e-01]
 [8.56878771e-01]]


(array([[[3.09577057e-04],
         [2.83417305e-04],
         [2.48533637e-04],
         ...,
         [2.60346157e-04],
         [2.62051565e-04],
         [4.08300297e-04]],
 
        [[2.83417305e-04],
         [2.48533637e-04],
         [2.59433647e-04],
         ...,
         [2.66308008e-04],
         [2.65962550e-04],
         [4.08082143e-04]],
 
        [[2.48533637e-04],
         [2.59433647e-04],
         [2.72517408e-04],
         ...,
         [2.70282517e-04],
         [2.56185003e-04],
         [4.08033925e-04]],
 
        ...,
 
        [[9.38103352e-01],
         [9.47342355e-01],
         [9.55521246e-01],
         ...,
         [9.83849540e-01],
         [9.71984111e-01],
         [7.92238986e-01]],
 
        [[9.47342355e-01],
         [9.55521246e-01],
         [9.56076598e-01],
         ...,
         [9.87771823e-01],
         [9.56182550e-01],
         [8.08066238e-01]],
 
        [[9.55521246e-01],
         [9.56076598e-01],
         [9.50573593e-01],
         